<a href="https://colab.research.google.com/github/peritopaulomax/PRNU/blob/master/PRNU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementação do algoritmo descrito em  *M. Goljan, T. Filler, and J. Fridrich. Large Scale Test of Sensor Fingerprint Camera Identification. In N.D. Memon and E.J. Delp and P.W. Wong and J. Dittmann, editors, Proc. of SPIE, Electronic Imaging, Media Forensics and Security XI, volume 7254, January 2009*.

O código abaixo foi obtido de https://dde.binghamton.edu/download/camera_fingerprint/ e está aqui disponível para fins **acadêmicos e de pesquisa apenas**. Observe que o código está protegido pelas leis de direitos autorais dos Estados Unidos, sendo licenciado sob uma Licença Internacional Creative Commons Atribuição-NãoComercial 4.0, vedando o uso do código para fins comerciais sem a permissão expressa por escrito do titular dos direitos autorais. Para obter permissão, entre em contato com Scott Moser em smoser@binghamton.edu.

Adaptação do códio para uso em notebook python com colaboração de:

1) Andrea Alves Guimarães Dresch

2) Paulo Max Gil Innocencio Reis

# **Importação das bibliotecas e funções necessárias. Bibliteca de funções deve estar carregada na pasta "*src*"**

In [3]:
import os

if os.path.exists('/content/PRNU'):
  !rm -rf {'/content/PRNU'}

if not os.path.exists('/content/PRNU'):
  !git clone https://github.com/peritopaulomax/PRNU.git


if not os.path.exists('/content/src'):
  !cp -R /content/PRNU/src /content/src

if not os.path.exists('/content/padrao'):
  !mkdir /content/padrao

import src.Functions as Fu
import src.Filter as Ft
import src.getFingerprint as gF
import src.maindir as md
import src.extraUtils as eu
import numpy as np
import cv2 as cv
import pickle
import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import scipy as sp



Cloning into 'PRNU'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 25 (delta 7), reused 19 (delta 5), pack-reused 0
Receiving objects: 100% (25/25), 36.53 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (7/7), done.


# **Utilize as celulas abaixo caso deseje calcular o PRNU de uma câmera alegada a partir de imagens padrão**



Definição dos caminhos da pasta contendo o material padrão (imagens coletadas a partir da câmera conhecida), As imagens padrão devem ser as mais homogêneas possivel (sem conteúdo em alta frequência), e submetidas a luminãncia elevada, sem saturação.

In [2]:
caminho_pasta = '/content/padrao'
lista = os.listdir(caminho_pasta)
padrao = [os.path.join(caminho_pasta, x) for x in lista]

Cálculo do padrão de PRNU da câmera que alegadamente é fonte da imagem questionada a partir das imagens padrão.

In [3]:
RP,_,_ = gF.getFingerprint(padrao)
RP = Fu.rgb2gray1(RP)
sigmaRP = np.std(RP)
Fingerprint = Fu.WienerInDFT(RP, sigmaRP)
nome_arquivo_fingerprint = input("Digite o nome do arquivo para salvar PRNU: ")
if not nome_arquivo_fingerprint.endswith(".prnu"):
    nome_arquivo_fingerprint += ".prnu"

with open(nome_arquivo_fingerprint, 'wb') as arquivo:
    pickle.dump(Fingerprint, arquivo)


Digite o nome do arquivo para salvar PRNU: camera1


# **Utilize a célula abaixo caso deseje carregar um arquivo PRNU calculado anteriormente calculado**

  * Não é necessario recarregar se tiver acabado de gerar o arquivo PRNU no passo anterior


In [4]:
with open('/content/camera1.prnu', 'rb') as arquivo:
    Fingerprint = pickle.load(arquivo)

# **Extração do resíduo de ruído da imagem questionada para comparação com o padrão PRNU da câmera alegada**

In [5]:
questionado = '/content/Ex-2_rec_escala.png'
Noisex = Ft.NoiseExtractFromImage(questionado, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))


# **Escolha do modo de comparação entre padrão e questionado**

In [6]:
def tratar_selecao(opcao):
  global selecao_radiobutton
  selecao_radiobutton = opcao
  print("Opção selecionada:", opcao)
# Criação da combobox com as opções
opcoes = ['Completa', 'Recortada', 'Recortada e Redimensionada']
radiobutton = widgets.RadioButtons(options=opcoes)

# Exibição da combobox
#display(radiobutton)
widgets.interact(tratar_selecao, opcao=radiobutton)


interactive(children=(RadioButtons(description='opcao', options=('Completa', 'Recortada', 'Recortada e Redimen…

<function __main__.tratar_selecao(opcao)>

# **Comparação entre questionado e padrão**

In [ ]:
Ix = cv.cvtColor(cv.imread(questionado),# image in BGR format
                 cv.COLOR_BGR2GRAY)

if selecao_radiobutton == "Recortada":
  A=Noisex
  B=Ix
  if Fingerprint.shape[0] > A.shape[0] or Fingerprint.shape[1] > A.shape[1]:
    A = np.pad(A, [(0, abs(Fingerprint.shape[0]-A.shape[0])), (0, abs(Fingerprint.shape[1]-A.shape[1]))])
    B = np.pad(B, [(0, abs(Fingerprint.shape[0]-B.shape[0])), (0, abs(Fingerprint.shape[1]-B.shape[1]))])
  C = Fu.crosscorr(A, B*Fingerprint)
  det, det0 = md.PCE(C, [Fingerprint.shape[0] - Noisex.shape[0], Fingerprint.shape[1] - Noisex.shape[1]])
elif selecao_radiobutton == "Completa":
    aux = Fingerprint
    if Ix.shape[0] > Ix.shape[1]:
        Ix = np.rot90(Ix,-1)
        Noisex = np.rot90(Noisex,-1)

    if Fingerprint.shape[1] > Ix.shape[1]:
        Fingerprint = cv.resize(Fingerprint, np.size(Ix), interpolation=cv.INTER_LINEAR)

    if np.size(Ix) != np.size(Fingerprint):
        raise ValueError("PRNU e Questionada com dimensões diferentes. Verifique outro método")

    C= Fu.crosscorr(Noisex,np.multiply(Ix, Fingerprint))
    det, det0 = md.PCE(C)
    Fingerprint = aux
elif selecao_radiobutton == "Recortada e Redimensionada":
    detpce = 0
    rT = 1
    locT = [0, 0]
    rmax = 1 / max([Noisex.shape[0] / Fingerprint.shape[0], Noisex.shape[1]/ Fingerprint.shape[1]])
    r = 1
    i = 0
    pcepce=[]
    aux=Fingerprint
    for r in np.arange(rmax - round((rmax - 0.1) / 0.005) * 0.005, rmax, 0.005):
        # waitbar(r / rmax)  # Não há equivalente direto para waitbar em Python
        pass
        #Nsized = cv.resize(Noisex, None, fx=r, fy=r, interpolation=cv.INTER_LANCZOS4)
        #Ixsized = cv.resize(Ix, None, fx=r, fy=r, interpolation=cv.INTER_LANCZOS4)
        Fingerprint = cv.resize(aux, None, fx=1/r, fy=1/r, interpolation=cv.INTER_CUBIC)
        #A=Nsized
        #B=Ixsized
        A=Noisex
        B=Ix
        if Fingerprint.shape[0] > A.shape[0] or Fingerprint.shape[1] > A.shape[1]:
          A = np.pad(A, [(0, abs(Fingerprint.shape[0]-A.shape[0])), (0, abs(Fingerprint.shape[1]-A.shape[1]))])
          B = np.pad(B, [(0, abs(Fingerprint.shape[0]-B.shape[0])), (0, abs(Fingerprint.shape[1]-B.shape[1]))])
        C = Fu.crosscorr(A, B*Fingerprint)
        #if min([Fingerprint.shape[0] - Nsized.shape[0], Fingerprint.shape[1] - Nsized.shape[1]]) >= 0:
        if min([Fingerprint.shape[0] - Noisex.shape[0], Fingerprint.shape[1] - Noisex.shape[1]]) >= 0:
            #det, det0 = md.PCE(C, [Fingerprint.shape[0] - Nsized.shape[0], Fingerprint.shape[1] - Nsized.shape[1]])
            det, det0 = md.PCE(C, [Fingerprint.shape[0] - Noisex.shape[0], Fingerprint.shape[1] - Noisex.shape[1]])
            locT = det['PeakLocation']
            #C =  Fu.crosscorr(Nsized, Ixsized * Fingerprint[locT[0]:locT[0]+np.size(Nsized, 0), locT[1]:locT[1]+np.size(Nsized, 1)])
            #det, det0 = md.PCE(C)
            pcepce.append(det['PCE'])
            if det['PCE'] > detpce:
                detpce = det['PCE']
                rT = r
        i += 1

    print(1/rT)
    print(locT)











In [35]:
for key in det.keys(): print("{0}: {1}".format(key, det[key]))
#Z =np.fft.fftshift(C)
#X = np.arange(Z.shape[0])
#Y = np.arange(Z.shape[1])
#X, Y = np.meshgrid(X, Y)
#fig = plt.figure(figsize=(12, 8))
#ax = fig.add_subplot(111, projection='3d')
#ax.set_facecolor((0.7, 0.7, 0.7))
#cmap = LinearSegmentedColormap.from_list(
#    "mycmap", [(0, "blue"),(0.4, "white"), (0.7, "green"),(1, "red")]
#)
#ax.contour3D(X, Y, Z, 200, cmap=cmap)
#ax.set_xlabel('X')
#ax.set_ylabel('Y')
#ax.set_zlabel('Z')
#plt.show()

PCE: 9.862630363509002
pvalue: 0.0008433489075323214
PeakLocation: [3, 2]
peakheight: 0.003384115311269983
P_FA: 0.32890607393760535
log10P_FA: -0.4829281062886186


In [ ]:
ax.set_facecolor((0.1, 0.1, 0.1))

In [ ]:
plt.show()

In [ ]:
!ls src/ -a
!rmdir src


In [14]:
np.max(C)

0.009750807362288992